In [6]:
import requests
import pandas as pd
import numpy as np
import json
import urllib

# We create a function to access Foursquare API and store the results in a dataframe and a CSV

In [2]:
def places_search(term,street, zipcode):
    dict = {"near": zipcode,
            "address": street,
            "query" :term,
            #"intent":"match",
            "zip":zipcode,
            "limit":1,
            "client_id" : "",
            "client_secret" : "",
            "v" : "20181122"
           }
    params =urllib.parse.urlencode(dict)
    url = "https://api.foursquare.com/v2/venues/search?"+params
    resp = requests.get(url)
    #grabbing the JSON result
    data = json.loads(resp.text)
    
    return data


In [15]:
df_restaurant = pd.read_csv("restaurants_raw_data_foursquare.csv")
df_restaurant = df_restaurant[df_restaurant["dba"].notnull()]
count =1
#df_restaurant["searched_foursquare"] = False
df_restaurant = df_restaurant.sort_values(by=['searched_foursquare',"dba"],ascending = True)
print(len(df_restaurant))
print(len(df_restaurant[df_restaurant["searched_foursquare"] == True ]))


for restau in df_restaurant.index[0:2000]:
    if df_restaurant.loc[restau,"searched_foursquare"] == False:
        jsonData = places_search(str(df_restaurant.loc[restau,"dba"]),str(df_restaurant.loc[restau,"street"]),str(df_restaurant.loc[restau,"zipcode"].astype(int)))
        try:
            df_restaurant.loc[restau,"foursquare_id"] = jsonData["response"]["venues"][0]["id"]
            df_restaurant.loc[restau,"foursquare_name"] = jsonData["response"]["venues"][0]["name"]
            df_restaurant.loc[restau,"foursquare_address"] = jsonData["response"]["venues"][0]["location"]["address"]
            df_restaurant.loc[restau,"foursquare_lat"] = jsonData["response"]["venues"][0]["location"]["lat"]
            df_restaurant.loc[restau,"foursquare_lng"] = jsonData["response"]["venues"][0]["location"]["lng"]
            df_restaurant.loc[restau,"foursquare_category_id"] = jsonData["response"]["venues"][0]["categories"][0]["id"]
            df_restaurant.loc[restau,"foursquare_category_name"] = jsonData["response"]["venues"][0]["categories"][0]["name"]
            print(count,str(df_restaurant.loc[restau,"dba"]),jsonData["response"]["venues"][0]["name"])
        except:
            print("Foursquare couldn't find!")
        df_restaurant.loc[restau,"searched_foursquare"] = True
        count = count +1    
        df_restaurant.to_csv("restaurants_raw_data_foursquare.csv",index=False)

26533
26533
